In [103]:
import pandas as pd
import numpy as np
import nltk
import string
import sklearn
from nltk.corpus import stopwords
from string import punctuation
stopwords = set(stopwords.words('english'))

In [105]:
vox = pd.read_json('./data/vox.jsonl', lines=True)

In [106]:
vox['id'] = vox._id.apply(lambda x: x['$oid'])
vox.drop('_id', axis=1, inplace=True)
trans = str.maketrans('', '', string.punctuation + '0123456789')
vox.text = vox.text.apply(lambda x: list(filter(lambda w: w not in stopwords, nltk.word_tokenize(x.lower().translate(trans)))))
vox.text = vox.text.apply(lambda x: list(filter(lambda y: len(y) > 2, x)))
vox.text = vox.text.apply(lambda x: list(map(lambda y: y.encode('ascii', 'ignore'), x)))

In [106]:
y = vox.date.apply(lambda x: x[:-1] + 'AM' if x.endswith('a') else x[:-1] + 'PM' if x.endswith('p') else x)
y = y.apply(lambda x: x[:12] if x.endswith('M') else x)

for row in y.iteritems():
    try:
        if row[1] == 'NULL':
            y[row[0]] = np.nan
        else:
            y[row[0]] = pd.to_datetime(row[1])
    except:
        pass

vox.date = y

In [107]:
from sklearn.feature_extraction.text import CountVectorizer

cnt_vct = CountVectorizer()
tokens = cnt_vct.fit_transform(vox.text)

AttributeError: 'list' object has no attribute 'lower'